##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 ML Metadata 改进机器学习工程


<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://tensorflow.google.cn/tfx/tutorials/mlmd/mlmd_tutorial"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 TensorFlow.org 上查看</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/tfx/tutorials/mlmd/mlmd_tutorial.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 中运行</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/tfx/tutorials/mlmd/mlmd_tutorial.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 GitHub 上查看源代码</a>
</td>
<td><a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/tfx/tutorials/mlmd/mlmd_tutorial.ipynb"> <img width="32px" src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a></td>
  
</table>

假设您设置了一个生产机器学习流水线来对企鹅进行分类。流水线提取您的训练数据，训练并评估模型，然后将其推送到生产环境。

但是，当您稍后尝试将此模型与包含不同种类企鹅的更大数据集一起使用时，您会发现您的模型的行为与预期不符，并开始错误地对物种进行分类。

此时，您有兴趣了解：

- 当唯一可用的工件是生产中的模型时，调试模型的最高效方式是什么？
- 哪个训练数据集用于训练模型？
- 哪个训练运行导致了这个错误的模型？
- 模型评估结果在哪里？
- 从哪里开始调试？

[ML Metadata (MLMD)](https://github.com/google/ml-metadata) 是一个利用与机器学习模型关联的元数据来帮助您回答这些问题以及更多问题的库。一个实用的类比是将此元数据视为软件开发中的日志记录。利用 MLMD，您能够可靠地跟踪与机器学习流水线的各个组件相关联的工件和沿袭。

在本教程中，您将设置一个 TFX 流水线来创建一个模型，该模型根据企鹅的体重、嘴峰的长度和深度以及脚蹼长度将企鹅分类为三个物种。然后，您将使用 MLMD 跟踪流水线组件的沿袭。

## Colab 中的 TFX 流水线

Colab 是一个与生产环境显著不同的轻量级开发环境。在生产环境中，您可能拥有跨多个分布式系统的各种流水线组件，例如数据提取、转换、模型训练、运行历史记录等。对于本教程，您应该知道编排和元数据存储存在显著差异 – 它们都在 Colab 中本地处理。在[此处](https://tensorflow.google.cn/tfx/tutorials/tfx/components_keras#background)详细了解 Colab 中的 TFX。


## 安装

首先，我们安装并导入必要的软件包，设置路径，并下载数据。

### 升级 Pip

为了避免在本地运行时升级系统中的 Pip，请检查以确保在 Colab 中运行。当然，可以对本地系统单独升级。

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### 安装并导入 TFX

In [ ]:
 !pip install -q tfx

### 导入软件包

#### 是否已重新启动运行时？

如果您使用的是 Google Colab，则在首次运行上面的代码单元时必须重新启动运行时，方法是点击上面的“RESTART RUNTIME”按钮或使用“Runtime &gt; Restart runtime ...”菜单。这样做的原因是 Colab 加载软件包的方式。

In [ ]:
import os
import tempfile
import urllib
import pandas as pd

import tensorflow_model_analysis as tfma
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

检查 TFX 和 MLMD 版本。

In [ ]:
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))
import ml_metadata as mlmd
print('MLMD version: {}'.format(mlmd.__version__))

## 下载数据集

在此 colab 中，我们使用可以在 [Github](https://github.com/allisonhorst/palmerpenguins) 上找到的 [Palmer Penguins 数据集](https://allisonhorst.github.io/palmerpenguins/articles/intro.html)。我们通过忽略任何不完整的记录来处理数据集，丢弃 `island` 和 `sex` 列，并将标签转换为 `int32`。此数据集包含 334 条关于企鹅的体重、嘴峰的长度和深度及其脚蹼长度的记录。您使用此数据将企鹅分类为三个物种之一。

In [ ]:
DATA_PATH = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/data/labelled/penguins_processed.csv'
_data_root = tempfile.mkdtemp(prefix='tfx-data')
_data_filepath = os.path.join(_data_root, "penguins_processed.csv")
urllib.request.urlretrieve(DATA_PATH, _data_filepath)

## 创建 InteractiveContext

要在此笔记本中以交互方式运行 TFX 组件，请创建一个 `InteractiveContext`。`InteractiveContext` 使用带有临时 MLMD 数据库实例的临时目录。请注意，对 `InteractiveContext` 的调用在 Colab 环境之外是无运算的。

通常，最好将类似的流水线运行在 `Context` 下并为一组。

In [ ]:
interactive_context = InteractiveContext()

## 构造 TFX 流水线

TFX 流水线由多个组件构成，这些组件执行机器学习工作流的不同方面。在此笔记本中，您将创建并运行`ExampleGen`、`StatisticsGen`、`SchemaGen` 和 `Trainer` 组件，并使用 `Evaluator` 和 `Pusher` 组件来评估和推送经过训练的模型。

有关 TFX 流水线组件的更多信息，请参阅[组件教程](https://tensorflow.google.cn/tfx/tutorials/tfx/components_keras)。

注：通过设置各个组件来构建 TFX 流水线涉及大量样板代码。就本教程而言，如果您不完全理解流水线设置中的每一行代码，那也没关系。 

### 实例化并运行 ExampleGen 组件

In [ ]:
example_gen = tfx.components.CsvExampleGen(input_base=_data_root)
interactive_context.run(example_gen)

### 实例化并运行 StatisticsGen 组件

In [ ]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
interactive_context.run(statistics_gen)

### 实例化并运行 SchemaGen 组件

In [ ]:
infer_schema = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)
interactive_context.run(infer_schema)

### 实例化并运行 Trainer 组件


In [ ]:
# Define the module file for the Trainer component
trainer_module_file = 'penguin_trainer.py'

In [ ]:
%%writefile {trainer_module_file}

# Define the training algorithm for the Trainer module file
import os
from typing import List, Text

import tensorflow as tf
from tensorflow import keras

from tfx import v1 as tfx
from tfx_bsl.public import tfxio

from tensorflow_metadata.proto.v0 import schema_pb2

# Features used for classification - culmen length and depth, flipper length,
# body mass, and species.

_LABEL_KEY = 'species'

_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]


def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              schema: schema_pb2.Schema, batch_size: int) -> tf.data.Dataset:
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY), schema).repeat()


def _build_keras_model():
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  d = keras.layers.Dense(8, activation='relu')(d)
  d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)
  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])
  return model


def run_fn(fn_args: tfx.components.FnArgs):
  schema = schema_pb2.Schema()
  tfx.utils.parse_pbtxt_file(fn_args.schema_path, schema)
  train_dataset = _input_fn(
      fn_args.train_files, fn_args.data_accessor, schema, batch_size=10)
  eval_dataset = _input_fn(
      fn_args.eval_files, fn_args.data_accessor, schema, batch_size=10)
  model = _build_keras_model()
  model.fit(
      train_dataset,
      epochs=int(fn_args.train_steps / 20),
      steps_per_epoch=20,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)
  model.save(fn_args.serving_model_dir, save_format='tf')

运行 `Trainer` 组件。

In [ ]:
trainer = tfx.components.Trainer(
    module_file=os.path.abspath(trainer_module_file),
    examples=example_gen.outputs['examples'],
    schema=infer_schema.outputs['schema'],
    train_args=tfx.proto.TrainArgs(num_steps=100),
    eval_args=tfx.proto.EvalArgs(num_steps=50))
interactive_context.run(trainer)

### 评估并推送模型

在使用 `Pusher` 组件将模型推送到提供目录之前，请使用 `Evaluator` 组件评估和“祝福”模型。

In [ ]:
_serving_model_dir = os.path.join(tempfile.mkdtemp(),
                                  'serving_model/penguins_classification')

In [ ]:
eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(label_key='species', signature_name='serving_default')
    ],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(
                class_name='SparseCategoricalAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value': 0.6})))
        ])
    ],
    slicing_specs=[tfma.SlicingSpec()])

In [ ]:
evaluator = tfx.components.Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    schema=infer_schema.outputs['schema'],
    eval_config=eval_config)
interactive_context.run(evaluator)

In [ ]:
pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=tfx.proto.PushDestination(
        filesystem=tfx.proto.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
interactive_context.run(pusher)

运行 TFX 流水线会填充 MLMD 数据库。在下一部分中，您将使用 MLMD API 查询此数据库以获取元数据信息。

## 查询 MLMD 数据库

MLMD 数据库存储三种类型的元数据：

- 有关流水线的元数据以及与流水线组件关联的沿袭信息
- 有关在流水线运行期间生成的工件的元数据
- 有关流水线执行的元数据

当新数据到达时，典型的生产环境流水线为多个模型提供服务。当您在提供的模型中遇到错误结果时，您可以查询 MLMD 数据库以隔离错误模型。然后，您可以跟踪与这些模型对应的流水线组件的沿袭来调试您的模型

使用之前定义的 `InteractiveContext` 设置元数据 (MD) 存储以查询 MLMD 数据库。

In [ ]:
connection_config = interactive_context.metadata_connection_config
store = mlmd.MetadataStore(connection_config)

# All TFX artifacts are stored in the base directory
base_dir = connection_config.sqlite.filename_uri.split('metadata.sqlite')[0]

创建一些辅助函数来查看 MD 存储中的数据。

In [ ]:
def display_types(types):
  # Helper function to render dataframes for the artifact and execution types
  table = {'id': [], 'name': []}
  for a_type in types:
    table['id'].append(a_type.id)
    table['name'].append(a_type.name)
  return pd.DataFrame(data=table)

In [ ]:
def display_artifacts(store, artifacts):
  # Helper function to render dataframes for the input artifacts
  table = {'artifact id': [], 'type': [], 'uri': []}
  for a in artifacts:
    table['artifact id'].append(a.id)
    artifact_type = store.get_artifact_types_by_id([a.type_id])[0]
    table['type'].append(artifact_type.name)
    table['uri'].append(a.uri.replace(base_dir, './'))
  return pd.DataFrame(data=table)

In [ ]:
def display_properties(store, node):
  # Helper function to render dataframes for artifact and execution properties
  table = {'property': [], 'value': []}
  for k, v in node.properties.items():
    table['property'].append(k)
    table['value'].append(
        v.string_value if v.HasField('string_value') else v.int_value)
  for k, v in node.custom_properties.items():
    table['property'].append(k)
    table['value'].append(
        v.string_value if v.HasField('string_value') else v.int_value)
  return pd.DataFrame(data=table)

首先，查询 MD 存储以获取其所有存储的 `ArtifactTypes` 的列表。

In [ ]:
display_types(store.get_artifact_types())

接下来，查询所有 `PushedModel` 工件。

In [ ]:
pushed_models = store.get_artifacts_by_type("PushedModel")
display_artifacts(store, pushed_models)

查询 MD 存储中最新推送的模型。本教程只有一个推送模型。 

In [ ]:
pushed_model = pushed_models[-1]
display_properties(store, pushed_model)

调试推送模型的第一步是查看推送了哪个训练模型，并查看使用哪些训练数据来训练该模型。

MLMD 提供遍历 API 来遍历溯源图，您可以使用它来分析模型溯源。 

In [ ]:
def get_one_hop_parent_artifacts(store, artifacts):
  # Get a list of artifacts within a 1-hop of the artifacts of interest
  artifact_ids = [artifact.id for artifact in artifacts]
  executions_ids = set(
      event.execution_id
      for event in store.get_events_by_artifact_ids(artifact_ids)
      if event.type == mlmd.proto.Event.OUTPUT)
  artifacts_ids = set(
      event.artifact_id
      for event in store.get_events_by_execution_ids(executions_ids)
      if event.type == mlmd.proto.Event.INPUT)
  return [artifact for artifact in store.get_artifacts_by_id(artifacts_ids)]

查询推送模型的父工件。

In [ ]:
parent_artifacts = get_one_hop_parent_artifacts(store, [pushed_model])
display_artifacts(store, parent_artifacts)

查询模型的属性。

In [ ]:
exported_model = parent_artifacts[0]
display_properties(store, exported_model)

查询模型的上游工件。

In [ ]:
model_parents = get_one_hop_parent_artifacts(store, [exported_model])
display_artifacts(store, model_parents)

获取训练模型所用的训练数据。

In [ ]:
used_data = model_parents[0]
display_properties(store, used_data)

现在，您获得了模型训练所使用的训练数据，再次查询数据库以找到训练步骤（执行）。查询 MD 存储以获取已注册执行类型的列表。

In [ ]:
display_types(store.get_execution_types())

训练步骤是名为 `tfx.components.trainer.component.Trainer` 的 `ExecutionType`。遍历 MD 存储，获取推送模型对应的 trainer 运行。

In [ ]:
def find_producer_execution(store, artifact):
  executions_ids = set(
      event.execution_id
      for event in store.get_events_by_artifact_ids([artifact.id])
      if event.type == mlmd.proto.Event.OUTPUT)
  return store.get_executions_by_id(executions_ids)[0]

trainer = find_producer_execution(store, exported_model)
display_properties(store, trainer)

## 总结

在本教程中，您学习了如何利用 MLMD 来跟踪 TFX 流水线组件的沿袭并解决问题。

要了解有关如何使用 MLMD 的更多信息，请查看以下附加资源：

- [MLMD API 文档](https://tensorflow.google.cn/tfx/ml_metadata/api_docs/python/mlmd)
- [MLMD 指南](https://tensorflow.google.cn/tfx/guide/mlmd)